# ДЗ Text classification using CNN

## Задача (Sentiment Analysis)

Собраны твиты 2-ух тональностей, необходимо произвести классификацию на 2-а класса.

In [2]:
max_words = 2000
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [4]:
import pandas as pd

df_train = pd.read_csv('/train.csv')
df_test = pd.read_csv("/test.csv")
df_val = pd.read_csv("/val.csv")

In [5]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [6]:
df_test.head()

,id,text
0,204150,Тектоника и рельеф-самое ужасное в мире мучение(
1,204151,"Ходили запускать шар желаний, но у нас не полу..."
2,204152,"Хочу лето только ради того, что бы направить н..."
3,204153,RT @RonyLiss: @colf_ne блин((\nа я шипперила Ф...
4,204154,"RT @anna_romt: @ZADROT_PO_IGRAM блин,каждое во..."


In [7]:
df_val.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


### Предобработка

In [9]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32917 sha256=4cfd13acb341a58b39a153d90ad2a1c8460b0cfcf82c10639ad5a2fd9706e061
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [10]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 7.1MB 6.4MB/s 


In [11]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [13]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [14]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [16]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [17]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [18]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [19]:
tokens_filtered_top[10:]

['вообще',
 'думать',
 'идти',
 'иня',
 'блин',
 'самый',
 'сидеть',
 'спать',
 'дом',
 '3',
 'друг',
 'писать',
 'сделать',
 'утро',
 'школа',
 '2',
 'ждать',
 'настроение',
 'мама',
 'ахи',
 'пойти',
 'любимый',
 'хотеться',
 'болеть',
 'написать',
 'видеть',
 'работа',
 'свой',
 'последний',
 'посмотреть',
 'добрый',
 'найти',
 'ночь',
 'понять',
 'купить',
 'понимать',
 'нравиться',
 'чтоть',
 'скучать',
 'скоро',
 'дать',
 'час',
 'вчера',
 'давать',
 'читать',
 'стать',
 'дело',
 'остаться',
 'забыть',
 'говорить',
 'работать',
 'плохо',
 'фильм',
 'урок',
 'оо',
 'жить',
 'ничегон',
 'вечер',
 'снег',
 'ходить',
 'домой',
 'пара',
 'сон',
 'нормальный',
 '5',
 'нужный',
 'начать',
 'нг',
 'телефон',
 'надеяться',
 'жаль',
 '4',
 'бля',
 'правда',
 'улица',
 'увидеть',
 'ахахи',
 'слово',
 'твиттереть',
 'изз',
 'вроде',
 'равно',
 'новогодний',
 'помнить',
 'рука',
 'большой',
 'дажена',
 'минута',
 'твит',
 'место',
 'фотка',
 'ехать',
 '1',
 'рождение',
 'играть',
 'половина',

In [20]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [21]:
import numpy as np
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [22]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [23]:
x_train.shape

(181467, 40)

In [24]:
x_train[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   1, 202, 437, 158,
         6], dtype=int32)

# Keras model

In [25]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [26]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)

In [42]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [43]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [44]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 8s 24ms/step - loss: 0.5744 - accuracy: 0.6813 - val_loss: 0.5499 - val_accuracy: 0.6999
Epoch 2/20
319/319 [==============================] - 8s 24ms/step - loss: 0.5327 - accuracy: 0.7164 - val_loss: 0.5470 - val_accuracy: 0.7015
Epoch 3/20
319/319 [==============================] - 8s 24ms/step - loss: 0.5066 - accuracy: 0.7360 - val_loss: 0.5500 - val_accuracy: 0.7021


In [45]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 9ms/step - loss: 0.5614 - accuracy: 0.6941


Test score: 0.5614053606987
Test accuracy: 0.6941321492195129


In [46]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

45/45 [==============================] - 0s 7ms/step


Теперь посмотрим word2vec

In [47]:
import nltk
nltk.download('punkt')
df_train['text_token'] = df_train['text'].apply(lambda x: nltk.tokenize.word_tokenize(x))
df_train['text_token'].head(2)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0    [alisachachkaн, уезжаааааааать, ❤, тожена, уез...
1    [rt, galyginvadim, ребята, девчата, кино, любо...
Name: text_token, dtype: object

In [68]:
max_len = 200

In [69]:
from gensim.models import Word2Vec
modelW2V = Word2Vec(sentences=df_train['text_token'], size=max_len, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34)

In [70]:
l = len(df_train['text_token'])
modelW2V.train([["hello", "world"]], total_examples=l, epochs=20)

(40, 40)

In [71]:
list(modelW2V.wv.vocab.keys())[:10]

['❤',
 'тожена',
 'уезжать',
 'rt',
 'galyginvadim',
 'ребята',
 'девчата',
 'кино',
 'любовь',
 'завтра']

In [72]:
def get_vector(tweet, model, size):
    vector = np.zeros(size)
    norm = 0
    for word in tweet:
        if word in model:
#            print(model[word])
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
#    print(vector)    
    return vector
    

In [63]:
def word2vec(text, max_len):
    padding = get_vector(text, modelW2V, max_len)        
    return padding

In [73]:
x_train = np.asarray([word2vec(text, max_len) for text in df_train["text"]], dtype=np.float)
x_test = np.asarray([word2vec(text, max_len) for text in df_test["text"]], dtype=np.float)
x_val = np.asarray([word2vec(text, max_len) for text in df_val["text"]], dtype=np.float)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [74]:
x_val[0]

array([-0.06971743,  0.11461692,  0.01857886,  0.17572134,  0.1337138 ,
       -0.06400263, -0.08668889, -0.23518825,  0.00785943, -0.03390469,
        0.21238295, -0.04811243, -0.12852715,  0.16513976, -0.15731257,
        0.1378894 , -0.09309374,  0.08760185,  0.1184424 ,  0.09868269,
        0.14519504,  0.16162618,  0.18734167, -0.03032083, -0.19047221,
       -0.18847665, -0.05904012,  0.06857269, -0.07616063,  0.08437547,
        0.02572098,  0.01014211, -0.00037681, -0.24608478, -0.00647513,
       -0.20026101,  0.00762361, -0.03444053,  0.23609251,  0.09397777,
        0.08381789,  0.02750923,  0.072346  , -0.1281656 , -0.04202159,
       -0.29754912,  0.06536306,  0.05637842,  0.20915617,  0.09762205,
       -0.09389511,  0.11918464,  0.1220331 , -0.17528061, -0.11039565,
       -0.06862528,  0.29002417, -0.09558262,  0.01054835,  0.08506566,
        0.01316793, -0.0038082 ,  0.17293618,  0.35604502,  0.10074459,
        0.14606528,  0.04433867, -0.2578047 , -0.00597849, -0.07

In [77]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [78]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [79]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 27s 85ms/step - loss: 0.6931 - accuracy: 0.5048 - val_loss: 0.6931 - val_accuracy: 0.5049
Epoch 2/20
319/319 [==============================] - 27s 85ms/step - loss: 0.6931 - accuracy: 0.5065 - val_loss: 0.6931 - val_accuracy: 0.5049


In [80]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 33ms/step - loss: 0.6931 - accuracy: 0.5047


Test score: 0.6931247115135193
Test accuracy: 0.5047392249107361


In [81]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

45/45 [==============================] - 1s 30ms/step


Вывод: в случае word2vec Test score=0.69, а было 0.56, но Test accuracy=0.5047 (было 0.69). score увеличился, а accuracy точность уменьшилась. 